In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import csv
import time
from collections import Counter
%matplotlib inline

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#tf.enable_eager_execution()

In [3]:
def create_data(filename):
    
    labels=[]
    feature=[]
    with open(filename,'r') as f:
        line = csv.reader(f)
        next(line,None)
        for row in line:
            feature.append([ np.int32(dictionary[x]) if x in dictionary else 0 for x in row[1].split()])
            labels.append(np.int32(row[0]))
    return feature, labels

In [4]:
def build_vocab(fileName):
    text=[]
    with open(fileName,'r') as f:
        line = csv.reader(f)
        next(line,None)
        for row in line:
            text.extend(row[1].replace('\n','<eos>').split())
    count=[['UNK',1]]
    count.extend(Counter(text).most_common())
    dictionary = dict()
    for word ,_ in count:
        dictionary[word]=len(dictionary)
    reverse_dictionary=dict(zip(dictionary.values(),dictionary.keys()))
    del text
    return dictionary,reverse_dictionary

In [5]:
def padd_row(arr,test=False):
    if test:
        max_len=424
    else:
        max_len=max(len(row) for row in arr)
    arr_padded = np.array([row + [0]*(max_len - len(row)) for row in arr])
    return arr_padded

In [6]:
dictionary,reverse_dictionary=build_vocab('data/train.csv')

In [7]:
len(dictionary)

15694

In [8]:
len(reverse_dictionary)

15694

In [9]:
feature,labels=create_data('data/train.csv')

In [10]:
def encode_labels(arr,):
    arr=arr
    for i in range(len(arr)):
        if arr[i]==5:
            arr[i]=1
            
        else:
            arr[i]=0
    return arr

In [11]:
labels=encode_labels(labels)

In [12]:
#feature=np.array(feature)

In [13]:
#feature_padd = padd_row(feature)

In [14]:
#len(feature_padd)

In [15]:
#x_data = tf.convert_to_tensor(feature_padd,dtype=tf.int32,name='feature')

In [16]:
#len(feature_padd)
#x_data[:100,0:20]

In [17]:
def create_batch(feature,label,batch_size,test=False):
    #assert feature.shape[0]%batch_size==0,"Use batch size in multiple of 10"
    feature_padd = padd_row(feature,test)
    #x_data = tf.convert_to_tensor(feature_padd,dtype=tf.int32,name='feature')
    #data_size = tf.size(x_data)
    no_batch = len(feature_padd)//batch_size
    #data = tf.reshape(x_data[:no_batch*batch_size],[batch_size,no_batch]) # or we can user [batch_size,-1]
    #labels = tf.convert_to_tensor(np.array(label),dtype=tf.int32,name='label')
    label=np.array(label)
    x_=feature_padd[:no_batch*batch_size]
    y_=label[:no_batch*batch_size]
    
    for n in range(0, len(x_), batch_size):
        #x = data[:,n:n+seq_n]
        x = x_[n:n+batch_size]
        y = y_[n:n+batch_size]
    
        yield x ,y   
    
    

In [18]:
batch = create_batch(feature,labels,50,424)
x,y=next(batch)
print (x.shape)
print (y.shape)
for e in range(20):
    for x,y in create_batch(feature,labels,100):
        print (x)
        print (y)

(50, 424)
(50,)
[[  20  322   85 ...    0    0    0]
 [   1  517   42 ...    0    0    0]
 [   1  330 8077 ...    0    0    0]
 ...
 [   1  346   35 ...    0    0    0]
 [ 734   26    5 ...    0    0    0]
 [ 279    0    0 ...    0    0    0]]
[1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 0 1 1 1 1 0 0 0 1 1 1 0 1 1 0 1 0 1 1
 0 0 1 1 1 1 1 1 0 1 1 1 1 0 0 1 1 1 1 0 1 1 0 1 0 1 1 0 1 1 1 1 1 1 1 1 1
 0 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 0 1 1 1]
[[   1 1541    8 ...    0    0    0]
 [2119 3222   27 ...    0    0    0]
 [ 279 1376    2 ...    0    0    0]
 ...
 [   1 2329  323 ...    0    0    0]
 [ 279 2155   73 ...    0    0    0]
 [   1  586 5950 ...    0    0    0]]
[1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 0 0 1 1 1 0 1 1 1 0
 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1 0 1 0 0 1 1 1 1
 0 0 1 1 1 1 1 1 1 1 1 1 0 0 0 1 1 1 1 1 1 1 1 0 1 1]
[[   1 1083 4559 ...    0    0    0]
 [1652 1991    2 ...    0    0    0]
 [  20  192    1 ...    0    0    0]
 ...
 [16

[[  20  322   85 ...    0    0    0]
 [   1  517   42 ...    0    0    0]
 [   1  330 8077 ...    0    0    0]
 ...
 [   1  346   35 ...    0    0    0]
 [ 734   26    5 ...    0    0    0]
 [ 279    0    0 ...    0    0    0]]
[1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 0 1 1 1 1 0 0 0 1 1 1 0 1 1 0 1 0 1 1
 0 0 1 1 1 1 1 1 0 1 1 1 1 0 0 1 1 1 1 0 1 1 0 1 0 1 1 0 1 1 1 1 1 1 1 1 1
 0 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 0 1 1 1]
[[   1 1541    8 ...    0    0    0]
 [2119 3222   27 ...    0    0    0]
 [ 279 1376    2 ...    0    0    0]
 ...
 [   1 2329  323 ...    0    0    0]
 [ 279 2155   73 ...    0    0    0]
 [   1  586 5950 ...    0    0    0]]
[1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 0 0 1 1 1 0 1 1 1 0
 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1 0 1 0 0 1 1 1 1
 0 0 1 1 1 1 1 1 1 1 1 1 0 0 0 1 1 1 1 1 1 1 1 0 1 1]
[[   1 1083 4559 ...    0    0    0]
 [1652 1991    2 ...    0    0    0]
 [  20  192    1 ...    0    0    0]
 ...
 [1669  168  203 ...

[[  20  322   85 ...    0    0    0]
 [   1  517   42 ...    0    0    0]
 [   1  330 8077 ...    0    0    0]
 ...
 [   1  346   35 ...    0    0    0]
 [ 734   26    5 ...    0    0    0]
 [ 279    0    0 ...    0    0    0]]
[1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 0 1 1 1 1 0 0 0 1 1 1 0 1 1 0 1 0 1 1
 0 0 1 1 1 1 1 1 0 1 1 1 1 0 0 1 1 1 1 0 1 1 0 1 0 1 1 0 1 1 1 1 1 1 1 1 1
 0 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 0 1 1 1]
[[   1 1541    8 ...    0    0    0]
 [2119 3222   27 ...    0    0    0]
 [ 279 1376    2 ...    0    0    0]
 ...
 [   1 2329  323 ...    0    0    0]
 [ 279 2155   73 ...    0    0    0]
 [   1  586 5950 ...    0    0    0]]
[1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 0 0 1 1 1 0 1 1 1 0
 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1 0 1 0 0 1 1 1 1
 0 0 1 1 1 1 1 1 1 1 1 1 0 0 0 1 1 1 1 1 1 1 1 0 1 1]
[[   1 1083 4559 ...    0    0    0]
 [1652 1991    2 ...    0    0    0]
 [  20  192    1 ...    0    0    0]
 ...
 [1669  168  203 ...

[[  20  322   85 ...    0    0    0]
 [   1  517   42 ...    0    0    0]
 [   1  330 8077 ...    0    0    0]
 ...
 [   1  346   35 ...    0    0    0]
 [ 734   26    5 ...    0    0    0]
 [ 279    0    0 ...    0    0    0]]
[1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 0 1 1 1 1 0 0 0 1 1 1 0 1 1 0 1 0 1 1
 0 0 1 1 1 1 1 1 0 1 1 1 1 0 0 1 1 1 1 0 1 1 0 1 0 1 1 0 1 1 1 1 1 1 1 1 1
 0 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 0 1 1 1]
[[   1 1541    8 ...    0    0    0]
 [2119 3222   27 ...    0    0    0]
 [ 279 1376    2 ...    0    0    0]
 ...
 [   1 2329  323 ...    0    0    0]
 [ 279 2155   73 ...    0    0    0]
 [   1  586 5950 ...    0    0    0]]
[1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 0 0 1 1 1 0 1 1 1 0
 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1 0 1 0 0 1 1 1 1
 0 0 1 1 1 1 1 1 1 1 1 1 0 0 0 1 1 1 1 1 1 1 1 0 1 1]
[[   1 1083 4559 ...    0    0    0]
 [1652 1991    2 ...    0    0    0]
 [  20  192    1 ...    0    0    0]
 ...
 [1669  168  203 ...

[[  20  322   85 ...    0    0    0]
 [   1  517   42 ...    0    0    0]
 [   1  330 8077 ...    0    0    0]
 ...
 [   1  346   35 ...    0    0    0]
 [ 734   26    5 ...    0    0    0]
 [ 279    0    0 ...    0    0    0]]
[1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 0 1 1 1 1 0 0 0 1 1 1 0 1 1 0 1 0 1 1
 0 0 1 1 1 1 1 1 0 1 1 1 1 0 0 1 1 1 1 0 1 1 0 1 0 1 1 0 1 1 1 1 1 1 1 1 1
 0 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 0 1 1 1]
[[   1 1541    8 ...    0    0    0]
 [2119 3222   27 ...    0    0    0]
 [ 279 1376    2 ...    0    0    0]
 ...
 [   1 2329  323 ...    0    0    0]
 [ 279 2155   73 ...    0    0    0]
 [   1  586 5950 ...    0    0    0]]
[1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 0 0 1 1 1 0 1 1 1 0
 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1 0 1 0 0 1 1 1 1
 0 0 1 1 1 1 1 1 1 1 1 1 0 0 0 1 1 1 1 1 1 1 1 0 1 1]
[[   1 1083 4559 ...    0    0    0]
 [1652 1991    2 ...    0    0    0]
 [  20  192    1 ...    0    0    0]
 ...
 [1669  168  203 ...

...

[[  20  322   85 ...    0    0    0]
 [   1  517   42 ...    0    0    0]
 [   1  330 8077 ...    0    0    0]
 ...
 [   1  346   35 ...    0    0    0]
 [ 734   26    5 ...    0    0    0]
 [ 279    0    0 ...    0    0    0]]
[1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 0 1 1 1 1 0 0 0 1 1 1 0 1 1 0 1 0 1 1
 0 0 1 1 1 1 1 1 0 1 1 1 1 0 0 1 1 1 1 0 1 1 0 1 0 1 1 0 1 1 1 1 1 1 1 1 1
 0 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 0 1 1 1]
[[   1 1541    8 ...    0    0    0]
 [2119 3222   27 ...    0    0    0]
 [ 279 1376    2 ...    0    0    0]
 ...
 [   1 2329  323 ...    0    0    0]
 [ 279 2155   73 ...    0    0    0]
 [   1  586 5950 ...    0    0    0]]
[1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 0 0 1 1 1 0 1 1 1 0
 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1 0 1 0 0 1 1 1 1
 0 0 1 1 1 1 1 1 1 1 1 1 0 0 0 1 1 1 1 1 1 1 1 0 1 1]
[[   1 1083 4559 ...    0    0    0]
 [1652 1991    2 ...    0    0    0]
 [  20  192    1 ...    0    0    0]
 ...
 [1669  168  203 ...

[[  20  322   85 ...    0    0    0]
 [   1  517   42 ...    0    0    0]
 [   1  330 8077 ...    0    0    0]
 ...
 [   1  346   35 ...    0    0    0]
 [ 734   26    5 ...    0    0    0]
 [ 279    0    0 ...    0    0    0]]
[1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 0 1 1 1 1 0 0 0 1 1 1 0 1 1 0 1 0 1 1
 0 0 1 1 1 1 1 1 0 1 1 1 1 0 0 1 1 1 1 0 1 1 0 1 0 1 1 0 1 1 1 1 1 1 1 1 1
 0 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 0 1 1 1]
[[   1 1541    8 ...    0    0    0]
 [2119 3222   27 ...    0    0    0]
 [ 279 1376    2 ...    0    0    0]
 ...
 [   1 2329  323 ...    0    0    0]
 [ 279 2155   73 ...    0    0    0]
 [   1  586 5950 ...    0    0    0]]
[1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 0 0 1 1 1 0 1 1 1 0
 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1 0 1 0 0 1 1 1 1
 0 0 1 1 1 1 1 1 1 1 1 1 0 0 0 1 1 1 1 1 1 1 1 0 1 1]
[[   1 1083 4559 ...    0    0    0]
 [1652 1991    2 ...    0    0    0]
 [  20  192    1 ...    0    0    0]
 ...
 [1669  168  203 ...

[[  20  322   85 ...    0    0    0]
 [   1  517   42 ...    0    0    0]
 [   1  330 8077 ...    0    0    0]
 ...
 [   1  346   35 ...    0    0    0]
 [ 734   26    5 ...    0    0    0]
 [ 279    0    0 ...    0    0    0]]
[1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 0 1 1 1 1 0 0 0 1 1 1 0 1 1 0 1 0 1 1
 0 0 1 1 1 1 1 1 0 1 1 1 1 0 0 1 1 1 1 0 1 1 0 1 0 1 1 0 1 1 1 1 1 1 1 1 1
 0 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 0 1 1 1]
[[   1 1541    8 ...    0    0    0]
 [2119 3222   27 ...    0    0    0]
 [ 279 1376    2 ...    0    0    0]
 ...
 [   1 2329  323 ...    0    0    0]
 [ 279 2155   73 ...    0    0    0]
 [   1  586 5950 ...    0    0    0]]
[1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 0 0 1 1 1 0 1 1 1 0
 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1 0 1 0 0 1 1 1 1
 0 0 1 1 1 1 1 1 1 1 1 1 0 0 0 1 1 1 1 1 1 1 1 0 1 1]
[[   1 1083 4559 ...    0    0    0]
 [1652 1991    2 ...    0    0    0]
 [  20  192    1 ...    0    0    0]
 ...
 [1669  168  203 ...

[[  20  322   85 ...    0    0    0]
 [   1  517   42 ...    0    0    0]
 [   1  330 8077 ...    0    0    0]
 ...
 [   1  346   35 ...    0    0    0]
 [ 734   26    5 ...    0    0    0]
 [ 279    0    0 ...    0    0    0]]
[1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 0 1 1 1 1 0 0 0 1 1 1 0 1 1 0 1 0 1 1
 0 0 1 1 1 1 1 1 0 1 1 1 1 0 0 1 1 1 1 0 1 1 0 1 0 1 1 0 1 1 1 1 1 1 1 1 1
 0 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 0 1 1 1]
[[   1 1541    8 ...    0    0    0]
 [2119 3222   27 ...    0    0    0]
 [ 279 1376    2 ...    0    0    0]
 ...
 [   1 2329  323 ...    0    0    0]
 [ 279 2155   73 ...    0    0    0]
 [   1  586 5950 ...    0    0    0]]
[1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 0 0 1 1 1 0 1 1 1 0
 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1 0 1 0 0 1 1 1 1
 0 0 1 1 1 1 1 1 1 1 1 1 0 0 0 1 1 1 1 1 1 1 1 0 1 1]
[[   1 1083 4559 ...    0    0    0]
 [1652 1991    2 ...    0    0    0]
 [  20  192    1 ...    0    0    0]
 ...
 [1669  168  203 ...

[[  20  322   85 ...    0    0    0]
 [   1  517   42 ...    0    0    0]
 [   1  330 8077 ...    0    0    0]
 ...
 [   1  346   35 ...    0    0    0]
 [ 734   26    5 ...    0    0    0]
 [ 279    0    0 ...    0    0    0]]
[1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 0 1 1 1 1 0 0 0 1 1 1 0 1 1 0 1 0 1 1
 0 0 1 1 1 1 1 1 0 1 1 1 1 0 0 1 1 1 1 0 1 1 0 1 0 1 1 0 1 1 1 1 1 1 1 1 1
 0 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 0 1 1 1]
[[   1 1541    8 ...    0    0    0]
 [2119 3222   27 ...    0    0    0]
 [ 279 1376    2 ...    0    0    0]
 ...
 [   1 2329  323 ...    0    0    0]
 [ 279 2155   73 ...    0    0    0]
 [   1  586 5950 ...    0    0    0]]
[1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 0 0 1 1 1 0 1 1 1 0
 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1 0 1 0 0 1 1 1 1
 0 0 1 1 1 1 1 1 1 1 1 1 0 0 0 1 1 1 1 1 1 1 1 0 1 1]
[[   1 1083 4559 ...    0    0    0]
 [1652 1991    2 ...    0    0    0]
 [  20  192    1 ...    0    0    0]
 ...
 [1669  168  203 ...

In [19]:
def create_cell(lstm_size,keep_prob):
    with tf.variable_scope("cells"):
        cell = tf.nn.rnn_cell.LSTMCell(lstm_size,reuse=tf.get_variable_scope().reuse)
        drop = tf.nn.rnn_cell.DropoutWrapper(cell,output_keep_prob=keep_prob)
    return drop

In [20]:
def build_input(batch_size,num_setps,num_classes):
    inputs = tf.placeholder(tf.int32,[batch_size,num_setps],name='input')
    targets = tf.placeholder(tf.int32,[batch_size],name='targets')
    keep_proba = tf.placeholder(tf.float32,name='keep_proba')
    
    return inputs,targets,keep_proba

In [21]:
def build_lstm(lstm_size,num_layers,batch_size,keep_proba):
    with tf.variable_scope("LSTM_Network"):
        cell = tf.contrib.rnn.MultiRNNCell([create_cell(lstm_size,keep_proba) for _ in range(num_layers)])
        initial_state = cell.zero_state(batch_size,tf.float32)
    
    return cell,initial_state

In [22]:
class YelpNetwork(object):
    def __init__(self,
                 lstm_size,
                 batch_size,
                 num_layers,
                 vocab,
                 numClasses=2,
                 num_steps=424,
                 learning_rate=0.001,
                 grad_clip=0.5,
                 embedding_size=300):
        
        self.lstm_size=lstm_size
        self.batch_size=batch_size
        self.num_layers=num_layers
        self.vocab_size=len(vocab)
        self.numClasses=numClasses
        self.grad_clip=grad_clip
        self.inputs,self.target,self.keep_proba=build_input(self.batch_size,num_steps,numClasses)
        self.learning_rate=learning_rate
        self.embedding= self.embedding_matrix(self.inputs,embedding_size,self.vocab_size)
        self.logits,self.out=self.build_output()
        self.loss=self.build_loss()
        self.optimizer=self.build_optimizer()
        self.accuracy =self.accuracy()
        
    
    def embedding_matrix(self,x,embedding_size,vocab_size):
        with tf.variable_scope('embedding'):
            embedd = tf.Variable(tf.random_uniform([vocab_size,embedding_size],-0.05,0.05))
            embedding=tf.nn.embedding_lookup(embedd,x)
        return embedding
    
    def build_output(self):
        
        with tf.variable_scope("dynamic_rnn",reuse=tf.AUTO_REUSE):
            cell,self.initial_state = build_lstm(self.lstm_size,self.num_layers,self.batch_size,self.keep_proba)
            output,state = tf.nn.dynamic_rnn(cell, self.embedding,initial_state=self.initial_state)
            self.final_state = state
        #output_flat = tf.reshape(output,[:,-1])
        
        with tf.variable_scope('softmax',reuse=tf.AUTO_REUSE):
            softmax_w = tf.Variable(tf.truncated_normal([self.lstm_size,self.numClasses],stddev=0.1))
            softmax_b = tf.Variable(tf.truncated_normal([self.numClasses],stddev=0.1))
        
        logits = tf.nn.xw_plus_b(output[:,-1],softmax_w,softmax_b)
        #logits = tf.sigmoid(logits,name='sigmoid')
        out = tf.nn.sigmoid(logits,name='predictions')
        #out = tf.argmax(out,axis=1)
        return logits,out
    
    def build_loss(self):
        
        with tf.variable_scope("loss"):
            
            y_one_hot = tf.one_hot(self.target,self.numClasses)
            #y_reshaped = tf.reshape(y_one_hot,(self.logits.get_shape()))
        
            loss = tf.nn.softmax_cross_entropy_with_logits(labels=y_one_hot,logits=self.logits)
            loss = tf.reduce_mean(loss)
        
        return loss
    
    def build_optimizer(self):
        
        with tf.variable_scope("optimizer"):
            tvars = tf.trainable_variables()
            grads,_ = tf.clip_by_global_norm(tf.gradients(self.loss,tvars),self.grad_clip)
            train_op = tf.train.AdamOptimizer(learning_rate=self.learning_rate)
            optimizer = train_op.apply_gradients(zip(grads,tvars))
        return optimizer
   
    def accuracy(self):
        with tf.variable_scope("accuracy"):
            #y_one_hot_test = tf.one_hot(self.target,self.numClasses)
            correct_predictions = tf.equal(tf.cast(tf.argmax(self.out,1),tf.int32),self.target)
            accuracy = tf.reduce_mean(tf.cast(correct_predictions,'float'))
        
        return accuracy
    
       

In [23]:
batch_size = 128         # Sequences per batch
#num_steps = 100          # Number of sequence steps per batch
lstm_size = 512         # Size of hidden layers in LSTMs
num_layers = 2          # Number of LSTM layers
#learning_rate = 0.001    # Learning rate
keep_prob = 0.5   

In [24]:
epoch = 4
save_every_n = 50

model = YelpNetwork(lstm_size,batch_size,num_layers,dictionary)

saver = tf.train.Saver(max_to_keep=100)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    # Use the line below to load a checkpoint and resume training
    #saver.restore(sess, 'checkpoints/______.ckpt')
    
    counter = 0
    
    for e in range (epoch):
        new_state = sess.run(model.initial_state)
        loss = 0.0
        for x,y in create_batch(feature,labels,batch_size):
        #while True:
            #x,y=next(create_batches(encoded, batch_size, num_steps))
            counter +=1
            start = time.time()
            feed = {model.inputs: x,
                    model.target: y,
                    model.keep_proba: keep_prob,
                    model.initial_state: new_state}
            
            batch_loss, new_state, _ = sess.run([model.loss, 
                                                 model.final_state, 
                                                 model.optimizer
                                                 ], 
                                                 feed_dict=feed)
            loss +=batch_loss
            end = time.time()
            print('Epoch: {}/{}... '.format(e+1, epoch),
                  'Training Step: {}... '.format(counter),
                  'Training loss: {:.4f}... '.format(batch_loss),
                  #'Training state: {:.4f}... '.format(new_state),
                  '{:.4f} sec/batch'.format((end-start)))
        
            if (counter % save_every_n == 0):
                saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))
    #print ('Out : ',out) 
    #print ('Logits : ',logits)
    saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))

Instructions for updating:
Use the retry module or similar alternatives.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

Epoch: 1/4...  Training Step: 1...  Training loss: 0.6598...  27.8760 sec/batch
Epoch: 1/4...  Training Step: 2...  Training loss: 0.5215...  29.6730 sec/batch
Epoch: 1/4...  Training Step: 3...  Training loss: 0.5493...  28.9830 sec/batch
Epoch: 1/4...  Training Step: 4...  Training loss: 0.4949...  28.6590 sec/batch
Epoch: 1/4...  Training Step: 5...  Training loss: 0.4625...  30.8520 sec/batch
Epoch: 1/4...  Training Step: 6...  Training loss: 0.4242...  27.4420 sec/batch
Epoch: 1/4...  Training Step: 7...  Training loss: 0.5098...  27.6970 sec/batch
Epoch: 1/4...  Training Step: 8...  Training loss: 0.4800...  27.3740 sec/batch
Epoch: 1/4...  Training Step: 9...  Training loss: 0.5634...  31.1880 sec/batch
Epoch: 1/4...  Tra

In [24]:
test_feature,test_labels=create_data('data/test.csv')

In [25]:
test_accu=[]
out_put=[]
model = YelpNetwork(lstm_size,batch_size,num_layers,dictionary)
saver=tf.train.Saver()
with tf.Session() as sess:
    saver.restore(sess,tf.train.latest_checkpoint('checkpoints'))
    #saver.restore(sess,'checkpoints\\i10_l512.ckpt')
    test_state=sess.run(model.initial_state)
    for x,y in create_batch(test_feature,test_labels,batch_size,test=True):
        feed = {model.inputs: x,
                model.target: y,
                model.keep_proba: 1,
                model.initial_state: test_state}
        accu,test_state,out=sess.run([model.accuracy,model.final_state,model.out],feed_dict=feed)
        test_accu.append(accu)
        out_put.extend(out)
        
    print('Test Accuracy: ',np.mean(test_accu))
        

Instructions for updating:
Use the retry module or similar alternatives.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

INFO:tensorflow:Restoring parameters from checkpoints\i84_l512.ckpt
Test Accuracy:  0.19453125


In [ ]:
out_put
